In [8]:
import pandas as pd 
df_article = pd.read_csv('/content/drive/My Drive/Project/BOK/ver1_bond_ngram.csv')
df_bond = pd.read_csv('/content/drive/My Drive/Project/BOK/ver2_article_ngram.csv')

In [9]:
df_article = df_article[['date', 'n_gram','Labelling']]
df_article = df_article.dropna(axis=0)

In [10]:
df_bond = df_bond.dropna(axis=0)
df_bond = df_bond[['date', 'n_gram','Labelling']]

In [29]:
df= pd.concat([df_bond, df_article], axis = 0, ignore_index=True) 
df

,date,n_gram,Labelling
0,2005-05-01,"['은행/NNG', '수수료/NNG', '수입/NNG', '급급/NNG', '일단/...",up
1,2005-05-01,"['선임/NNG', '고객가치/NNG', '증대/NNG', '시키/XSV', '판매...",up
2,2005-05-01,"['금리/NNG', '경쟁/NNG', '이자/NNG', '수익/NNG', '순이자마...",up
3,2005-05-01,"['투자/NNG', '상품/NNG', '둘러싸/VV', '판매/NNG', '경쟁/N...",up
4,2005-05-01,"['은행원/NNG', '고객/NNG', '제대로/MAG', '이해/NNG', '하/...",up
...,...,...,...
2971175,2008-05-22,"['속도조절/NNG', '중/NNG', '발행시장/NNG', '공모/NNG', '회...",up
2971176,2008-05-22,"['의사록/NNG', '확신/NNG', '없/VA', '인상/NNG', '의견/NN...",up
2971177,2008-05-22,"['금융/NNG', '수급/NNG', '영향/NNG', '점검/NNG', '금융/N...",up
2971178,2008-05-22,"['동향/NNG', '금리/NNG', '하락/NNG', '상승/NNG', '반전/N...",up


In [32]:
df['Labelling'].value_counts()

down    1238102
up      1093204
same     639874
Name: Labelling, dtype: int64

In [42]:
X = df['n_gram']
y = df['Labelling']

In [43]:
from sklearn.model_selection import train_test_split
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, stratify = y, test_size = 0.1, random_state = 1)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, stratify = y, test_size = 0.1, random_state = 2)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, stratify = y, test_size = 0.1, random_state = 3)
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, stratify = y, test_size = 0.1, random_state = 4)
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(X, y, stratify = y, test_size = 0.1, random_state = 5)

In [35]:
X_train.shape, y_train.shape, X_test.shape

((2674062,), (2674062,), (297118,))

In [74]:
y_train_1.reset_index()

,index,Labelling
0,652482,up
1,1707783,same
2,2898205,down
3,1864815,same
4,312402,down
...,...,...
2674057,2876481,up
2674058,418783,up
2674059,2646779,up
2674060,1001545,up


In [76]:
from tqdm.notebook import tqdm
from collections import defaultdict

def word_freq(X, y):
  doccnt0 = 0 
  doccnt1 = 0 
  X = X.reset_index()
  y = y.reset_index()
  wordfreq = defaultdict(lambda:[0,0])

  for i in tqdm(range(len(X))):
      w_list = []
      label = [0 if y['Labelling'].iloc[i] == 'up' else 1][0]
      w_list = X['n_gram'][i].strip('\[').replace("\'", "").strip('\]').split(",")
      for ngram in w_list:
            wordfreq[ngram][label] += 1
  return wordfreq

In [77]:
freq1 = word_freq(X_train_1, y_train_1)

In [82]:
freq2 = word_freq(X_train_2, y_train_2)

In [91]:
freq3 = word_freq(X_train_3, y_train_3)

In [92]:
freq4 = word_freq(X_train_4, y_train_4)

In [93]:
freq5 = word_freq(X_train_5, y_train_5)

In [78]:
def polarity(wordfreq):
  prob = pd.DataFrame(wordfreq.values(), columns=['up_count', 'down_count'])
  prob['ngram'] = pd.DataFrame(wordfreq.keys(), columns=['ngram'])
  prob['total_count'] = prob['up_count'] + prob['down_count']

  rule = prob[prob['total_count']<=15].index
  prob_ = prob.drop(rule)

  # 극성 계산
  haw_sum = sum(prob_['up_count'])
  dov_sum = sum(prob_['down_count'])

  prob_['polar_score'] = (prob_['up_count']/haw_sum) /(prob_['down_count']/dov_sum)

  prob_['polarity'] = prob_['polar_score'].apply(lambda x: 'Hawkish' if x> 1.3 else "Dovish" if x < 0.76 else "nothing")

  return prob_

In [88]:
train_1 = polarity(freq1)
train_1 = train_1[['ngram', 'polarity']]
train_1

,ngram,polarity
0,전경련/NNG,nothing
1,국제/NNG,nothing
2,회의장/NNG,Dovish
3,개최/NNG,nothing
4,하/VV,nothing
...,...,...
136831,채부/NNG,Hawkish
136832,기위/NNG,Hawkish
137491,금준/NNG,Hawkish
143379,문유/NNG,Hawkish


In [89]:
train_2 = polarity(freq2)
train_2 = train_2[['ngram', 'polarity']]
train_2

,ngram,polarity
0,또/MAG,nothing
1,대형화/NNG,Hawkish
2,마무리/NNG,nothing
3,되/XSV,nothing
4,되/VV,nothing
...,...,...
141153,투자/NNG;확대/NNG;성장/NNG,nothing
141918,박재/NNG,Hawkish
142143,축역/NNG,Hawkish
146329,문유/NNG,Hawkish


In [94]:
train_3 = polarity(freq3)
train_3 = train_3[['ngram', 'polarity']]
train_3

,ngram,polarity
0,일각/NNG,nothing
1,공포/NNG,Dovish
2,단어/NNG,nothing
3,쓰/VV,nothing
4,공화당/NNG,Dovish
...,...,...
132478,보수성/NNG,Hawkish
132902,산설/NNG,nothing
134061,투증/NNG,Hawkish
137093,선진국/NNG;재정/NNG;긴축/NNG,Dovish


In [95]:
train_4 = polarity(freq4)
train_4 = train_4[['ngram', 'polarity']]
train_4

,ngram,polarity
0,중국발/NNG,Hawkish
1,상품/NNG,nothing
2,가격/NNG,nothing
3,급락/NNG,nothing
4,미/NNG,nothing
...,...,...
130396,건강보험심사평가원/NNG,Dovish
132099,앆/NNG,Hawkish
134071,지수/NNG;충격/NNG,Dovish
138974,가격/NNG;둔화/NNG,Hawkish


In [96]:
train_5 = polarity(freq4)
train_5 = train_5[['ngram', 'polarity']]
train_5

,ngram,polarity
0,중국발/NNG,Hawkish
1,상품/NNG,nothing
2,가격/NNG,nothing
3,급락/NNG,nothing
4,미/NNG,nothing
...,...,...
130396,건강보험심사평가원/NNG,Dovish
132099,앆/NNG,Hawkish
134071,지수/NNG;충격/NNG,Dovish
138974,가격/NNG;둔화/NNG,Hawkish


In [97]:
onetwo = train_1.merge(train_2, left_on = 'ngram', right_on = 'ngram', how = 'outer' )
onetwo

,ngram,polarity_x,polarity_y
0,전경련/NNG,nothing,nothing
1,국제/NNG,nothing,nothing
2,회의장/NNG,Dovish,Dovish
3,개최/NNG,nothing,nothing
4,하/VV,nothing,nothing
...,...,...,...
48458,환경보호/NNG,NaN,nothing
48459,투자/NNG;마이너스/NNG,NaN,nothing
48460,나태/NNG,NaN,Dovish
48461,종적/NNG,NaN,Hawkish


In [98]:
onethree = onetwo.merge(train_3, left_on = 'ngram', right_on = 'ngram', how = 'outer' )
onethree

,ngram,polarity_x,polarity_y,polarity
0,전경련/NNG,nothing,nothing,nothing
1,국제/NNG,nothing,nothing,nothing
2,회의장/NNG,Dovish,Dovish,Dovish
3,개최/NNG,nothing,nothing,nothing
4,하/VV,nothing,nothing,nothing
...,...,...,...,...
49029,영업이익/NNG;하락/NNG,NaN,NaN,Hawkish
49030,대차대조표/NNG;줄/VV,NaN,NaN,nothing
49031,각계각층/NNG,NaN,NaN,Dovish
49032,퍼블릭/NNG,NaN,NaN,Hawkish


In [99]:
onefour = onethree.merge(train_4, left_on = 'ngram', right_on = 'ngram', how = 'outer' )
onefour

,ngram,polarity_x,polarity_y,polarity_x,polarity_y
0,전경련/NNG,nothing,nothing,nothing,nothing
1,국제/NNG,nothing,nothing,nothing,nothing
2,회의장/NNG,Dovish,Dovish,Dovish,Dovish
3,개최/NNG,nothing,nothing,nothing,nothing
4,하/VV,nothing,nothing,nothing,nothing
...,...,...,...,...,...
49389,경제/NNG;대외/NNG;위험/NNG,NaN,NaN,NaN,Hawkish
49390,차등가격/NNG,NaN,NaN,NaN,Hawkish
49391,검소/NNG,NaN,NaN,NaN,nothing
49392,기은/NNG,NaN,NaN,NaN,nothing


In [101]:
onefive = onefour.merge(train_5, left_on = 'ngram', right_on = 'ngram', how = 'outer' )
onefive

,ngram,polarity_x,polarity_y,polarity_x,polarity_y,polarity
0,전경련/NNG,nothing,nothing,nothing,nothing,nothing
1,국제/NNG,nothing,nothing,nothing,nothing,nothing
2,회의장/NNG,Dovish,Dovish,Dovish,Dovish,Dovish
3,개최/NNG,nothing,nothing,nothing,nothing,nothing
4,하/VV,nothing,nothing,nothing,nothing,nothing
...,...,...,...,...,...,...
49389,경제/NNG;대외/NNG;위험/NNG,NaN,NaN,NaN,Hawkish,Hawkish
49390,차등가격/NNG,NaN,NaN,NaN,Hawkish,Hawkish
49391,검소/NNG,NaN,NaN,NaN,nothing,nothing
49392,기은/NNG,NaN,NaN,NaN,nothing,nothing


In [109]:
onefive['freq'] = onefive.mode(axis=1)[0]
onefive

,ngram,polarity_x,polarity_y,polarity_x,polarity_y,polarity,freq
0,전경련/NNG,nothing,nothing,nothing,nothing,nothing,nothing
1,국제/NNG,nothing,nothing,nothing,nothing,nothing,nothing
2,회의장/NNG,Dovish,Dovish,Dovish,Dovish,Dovish,Dovish
3,개최/NNG,nothing,nothing,nothing,nothing,nothing,nothing
4,하/VV,nothing,nothing,nothing,nothing,nothing,nothing
...,...,...,...,...,...,...,...
49389,경제/NNG;대외/NNG;위험/NNG,NaN,NaN,NaN,Hawkish,Hawkish,Hawkish
49390,차등가격/NNG,NaN,NaN,NaN,Hawkish,Hawkish,Hawkish
49391,검소/NNG,NaN,NaN,NaN,nothing,nothing,nothing
49392,기은/NNG,NaN,NaN,NaN,nothing,nothing,nothing


In [110]:
df = onefive[['ngram', 'freq']]
df

,ngram,freq
0,전경련/NNG,nothing
1,국제/NNG,nothing
2,회의장/NNG,Dovish
3,개최/NNG,nothing
4,하/VV,nothing
...,...,...
49389,경제/NNG;대외/NNG;위험/NNG,Hawkish
49390,차등가격/NNG,Hawkish
49391,검소/NNG,nothing
49392,기은/NNG,nothing


In [122]:
df_hawkish_dict_ver4= df[df['freq']=='Hawkish']
df_hawkish_dict_ver4 

,ngram,freq
36,청/NNG,Hawkish
42,투자전략/NNG,Hawkish
54,기타공공기관/NNG,Hawkish
61,긴축/NNG,Hawkish
84,어떠/NNG,Hawkish
...,...,...
49384,다니엘/NNG,Hawkish
49385,고/XSV,Hawkish
49388,직장폐쇄/NNG,Hawkish
49389,경제/NNG;대외/NNG;위험/NNG,Hawkish


In [126]:
df_hawkish_dict_ver4.to_csv('df_hawkish_dict_ver5.csv', index=False)

In [124]:
df_dovish_dict_ver4 = df[df['freq']=='Dovish']
df_dovish_dict_ver4 

,ngram,freq
2,회의장/NNG,Dovish
29,irs/NNG,Dovish
50,양적완화/NNG,Dovish
59,ecb/NNG,Dovish
120,대부/NNG,Dovish
...,...,...
49376,거래처/NNG,Dovish
49379,주거권/NNG,Dovish
49380,금리/NNG;인상/NNG;전망/NNG;증가/NNG,Dovish
49381,배당률/NNG,Dovish


In [127]:
df_dovish_dict_ver4.to_csv('df_dovish_dict_ver5.csv', index=False)